# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Ana Sofía Hinojosa Bale  |
| **Fecha**      | 24 de agosto 2025        |
| **Expediente** | 742594                   |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv("Advertising.csv")
x = data.drop(columns = ['sales'])
y = data['sales']

### Modelos

In [4]:
lr = LinearRegression()
ridge = Ridge(alpha=0.1)
lasso = Lasso(alpha=0.1)

In [5]:
def resultado(model, x):
    scaler = StandardScaler().fit(x)
    x_scaled = scaler.transform(x)
    model.fit(x_scaled, y)
    y_pred = model.predict(x_scaled)
    n = x_scaled.shape[0]
    p = x_scaled.shape[1]
    RSS = np.sum((y_pred - y)**2)
    var = RSS / (n - p - 1)

    b_0 = model.intercept_
    b_i = model.coef_

    X = np.column_stack((np.ones(n), x_scaled))
    var_beta = np.linalg.inv(X.T @ X) * var
    std_beta = np.sqrt(np.diag(var_beta))

    t_stats = np.array([b_0] + list(b_i)) / std_beta
    p_values = [2 * (1 - stats.t.cdf(np.abs(t), n - p - 1)) for t in t_stats]

    if hasattr(x, 'columns'):
        nombres = ["Intercepto"] + list(x.columns)
    else:
        nombres = ["Intercepto"] + [f"x{i+1}" for i in range(x.shape[1])]

    return pd.DataFrame({
        'Parámetro': nombres,
        'Coeficiente': [b_0] + list(b_i),
        't-Estadistico': t_stats,
        'p-value': p_values
    }), pd.DataFrame({
        'R2': [r2_score(y, y_pred)]
    })

In [6]:
resultado(lr, x)[0]

,Parámetro,Coeficiente,t-Estadistico,p-value
0,Intercepto,14.022500,117.377059,0.000000
1,Unnamed: 0,-0.033476,-0.276207,0.782681
2,TV,3.920219,32.725344,0.000000
3,radio,2.789888,21.783554,0.000000
4,newspaper,-0.027010,-0.209600,0.834198


In [7]:
resultado(lasso, x)[0]

,Parámetro,Coeficiente,t-Estadistico,p-value
0,Intercepto,14.022500,116.947273,0.0
1,Unnamed: 0,-0.000000,-0.000000,1.0
2,TV,3.823608,31.801975,0.0
3,radio,2.689324,20.921458,0.0
4,newspaper,0.000000,0.000000,1.0


In [8]:
resultado(ridge, x)[0]

,Parámetro,Coeficiente,t-Estadistico,p-value
0,Intercepto,14.022500,117.376935,0.000000
1,Unnamed: 0,-0.033492,-0.276339,0.782580
2,TV,3.918307,32.709343,0.000000
3,radio,2.788364,21.771628,0.000000
4,newspaper,-0.026351,-0.204487,0.838186


In [9]:
pd.DataFrame({
    'Modelo': ['Linear Regression', 'Lasso', 'Ridge'],

    'R2': [
        resultado(lr, x)[1]['R2'][0],
        resultado(lasso, x)[1]['R2'][0],
        resultado(ridge, x)[1]['R2'][0]
    ]
})

,Modelo,R2
0,Linear Regression,0.897251
1,Lasso,0.896494
2,Ridge,0.897251
